Midterm Report:
Week 1
1. Benchmarks: Gemma 3 (4 Billion) and Llama 3 (Llama 3.2 3B)
2. Dataset: UCBerkely


Week 2:
1. Finetuning of Gemma 3 and Llama 3.2 3B with LoRA , and if necesasary QLoRA (maybe for endterm report). [Finetuning Benchmarks]
2. Evaluation of other opensource models with more parameters (low priority)


Week 3:
TBD
1. Start working on report and final exam.
2. Prompting strategies for Gemma 3 and Llama 3.2 3B which can improve performance.
3. Again perform evaluation on unfinetuned performed models with improved prompting strategies. (low priority)


After Midterm Exam:
1. We will focus on improving parameters of LoRA and QLoRA.
2. Also work on improvements suggested in midterm report.

<!-- # Proposed Schema

Input schema for finetuningb and ground truth:
```
{
  "overall": {"label": "none|abusive|hateful", "score": 0.0},
  "facets": {
    "insult": 0, "humiliate": 0, "dehumanize": 0,
    "violence": 0, "genocide": 0,
    "respect": 0, "status": 0, "attack_defend": 0, "sentiment": 0
  },
  "targets": {
    "race_asian": false, "race_black": false, "race_white": false, "race_latinx": false,
    "religion_muslim": false, "religion_jewish": false, "religion_hindu": false, "religion_buddhist": false,
    "gender_men": false, "gender_women": false, "gender_transgender": false, "gender_non_binary": false,
    "sexuality_gay": false, "sexuality_lesbian": false, "sexuality_bisexual": false, "sexuality_straight": false,
    "disability_physical": false, "disability_cognitive": false, "origin_immigrant": false, "other": false
  },
  "explanation" : "text",
}``` -->

# Gold Standard Dataset schema

```
{
  // --- Core Identifiers ---
  "comment_id": "131169.0", 
  "text": "The original text content of the social media comment.",

  // --- Holistic Overall Assessment ---
  "overall": {
    "label": "hateful",             // A categorical label derived from the paper's official thresholds.
                                    // - "hateful": If hate_speech_score > 0.5
                                    // - "supportive": If hate_speech_score < -1.0
                                    // - "neutral": If between -1.0 and 0.5 (inclusive)
    "hate_speech_score": 1.52       // Ground truth 
  },

  // --- Multi-Label Facet Analysis (The Core of the Task) ---
  "facets": {
    "sentiment": 3.0,               
    "respect": 3.0,                 
    "insult": 3.0,                  
    "humiliate": 3.0,
    "status": 0.0,
    "dehumanize": 3.0,
    "violence": 0.0,
    "genocide": 0.0,
    "attack_defend": 3.0,
    "hatespeech": 1.0               
  },

  // --- Multi-Label Target Group Identification ---
  "targets": {
    "race_asian": true,             // A dictionary of booleans. `true` means the comment targets this group.
    "race_black": false,            // This allows a single comment to target multiple groups simultaneously.
    "religion_muslim": true,
    "gender_women": false
    // ... and so on for all other possible target groups.
  }
}

In [1]:
import pandas as pd
import json

# Note: This script requires the following libraries to be installed:
# pip install pandas pyarrow fsspec huggingface_hub

# 1. Load the dataset into a pandas DataFrame (your preferred method)
print("Loading the dataset into a pandas DataFrame...")
try:
    df = pd.read_parquet("hf://datasets/ucberkeley-dlab/measuring-hate-speech/measuring-hate-speech.parquet")
    print(f"Dataset loaded successfully with {len(df)} rows and {len(df.columns)} columns.")
except Exception as e:
    print(f"An error occurred while loading the data: {e}")
    print("Please ensure you have run 'pip install pandas pyarrow fsspec huggingface_hub'")
    exit()

# 2. Define the columns we want to extract
# These are the actual facet and target columns present in the dataset
FACET_COLUMNS = [
    'sentiment', 'respect', 'insult', 'humiliate', 'status', 'dehumanize', 
    'violence', 'genocide', 'attack_defend', 'hatespeech'
]

TARGET_COLUMNS = [
    'target_race_asian', 'target_race_black', 'target_race_latinx', 
    'target_race_middle_eastern', 'target_race_native_american', 
    'target_race_pacific_islander', 'target_race_white', 'target_race_other',
    'target_religion_atheist', 'target_religion_buddhist', 'target_religion_christian',
    'target_religion_hindu', 'target_religion_jewish', 'target_religion_mormon',
    'target_religion_muslim', 'target_religion_other', 'target_origin_immigrant',
    'target_origin_migrant_worker', 'target_origin_specific_country',
    'target_origin_undocumented', 'target_origin_other', 'target_gender_men',
    'target_gender_non_binary', 'target_gender_transgender_men',
    'target_gender_transgender_unspecified', 'target_gender_transgender_women',
    'target_gender_women', 'target_gender_other', 'target_sexuality_bisexual',
    'target_sexuality_gay', 'target_sexuality_lesbian', 'target_sexuality_straight',
    'target_sexuality_other', 'target_disability_physical', 'target_disability_cognitive',
    'target_disability_neurological', 'target_disability_visually_impaired',
    'target_disability_hearing_impaired', 'target_disability_unspecific',
    'target_disability_other'
]

# Helper function to classify the score based on the paper's logic
def classify_score(score):
    if score > 0.5:
        return "hateful"
    if score < -1.0:
        return "supportive"
    return "neutral"

# 3. Create the function to transform each row of the DataFrame
def create_gold_standard_record(row):
    # Create the 'overall' object
    overall = {
        "label": classify_score(row['hate_speech_score']),
        "hate_speech_score": row['hate_speech_score']
    }
    
    # Create the 'facets' object
    facets = {col: row[col] for col in FACET_COLUMNS}
    
    # Create the 'targets' object (ensuring columns exist)
    targets = {col: bool(row[col]) for col in TARGET_COLUMNS if col in row}
    
    # Assemble the final record as a dictionary
    return {
        "comment_id": row['comment_id'],
        "text": row['text'],
        "overall": overall,
        "facets": facets,
        "targets": targets
    }

# 4. Apply the function to each row of the DataFrame
print("Applying the transformation to each row of the DataFrame...")
gold_records = df.apply(create_gold_standard_record, axis=1).tolist()

print("\nExample of a processed record:")
print(json.dumps(gold_records[0], indent=2))

output_file = "gold_benchmark_dataset.jsonl"
print(f"\nSaving the {len(gold_records)} records to '{output_file}'...")

with open(output_file, 'w') as f:
    for record in gold_records:
        f.write(json.dumps(record) + '\n')


Loading the dataset into a pandas DataFrame...


C:\Users\vinh2\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset loaded successfully with 135556 rows and 131 columns.
Applying the transformation to each row of the DataFrame...

Example of a processed record:
{
  "comment_id": 47777,
  "text": "Yes indeed. She sort of reminds me of the elder lady that played the part in the movie \"Titanic\" who was telling her story!!! And I wouldn't have wanted to cover who I really am!! I would be proud!!!! WE should be proud of our race no matter what it is!!",
  "overall": {
    "label": "supportive",
    "hate_speech_score": -3.9
  },
  "facets": {
    "sentiment": 0.0,
    "respect": 0.0,
    "insult": 0.0,
    "humiliate": 0.0,
    "status": 2.0,
    "dehumanize": 0.0,
    "violence": 0.0,
    "genocide": 0.0,
    "attack_defend": 0.0,
    "hatespeech": 0.0
  },
  "targets": {
    "target_race_asian": true,
    "target_race_black": true,
    "target_race_latinx": true,
    "target_race_middle_eastern": true,
    "target_race_native_american": true,
    "target_race_pacific_islander": true,
    "tar

# Next Steps: Benchmarking with unfinetuned models and calculate metrics against gold standard

Standardized prompt:

```You are an expert hate speech analyst. Your task is to analyze the provided text and return ONLY a valid JSON object that strictly follows the schema below. 
Do not include any explanations, markdown formatting, or text outside of the JSON object.

=========================
IMPORTANT INSTRUCTIONS
=========================
1. The output must be **valid JSON**, with no markdown or extra commentary.
2. Use the exact field names and types described below.
3. Each value must follow its correct type:
   - Floats → only for `score`
   - Integers → only for all values in `facets` (must be whole numbers, not floats)
   - Booleans → only for all values in `targets` (`true` or `false`, lowercase)
   - Strings → only for `label` (must be one of: "supportive", "neutral", "hateful")

=========================
OVERALL SCORE AND LABEL
=========================
Produce a single signed float named `"score"` and a string `"label"` inside `"overall"`.  

**Scoring Rules:**
- NEGATIVE float: `< -1` → Supportive content (e.g. `-1.35`)
- POSITIVE float: `> 0.5` → Hateful content (e.g. `1.47`)
- NEAR ZERO float: `-1 <= score <= 0.5` → Neutral content (e.g. `0.12`, `-0.08`)

**Label Rules:**
- If `score < -1`, then `"label": "supportive"`
- If `-1 <= score <= 0.5`, then `"label": "neutral"`
- If `score > 0.5`, then `"label": "hateful"`

=========================
FACETS (0-4 SCALE)
=========================
Each facet must be an **integer** from 0 to 4 (no decimals).  
Use this scale strictly:
- 0 = Absent  
- 1 = Mild  
- 2 = Clear  
- 3 = Severe  
- 4 = Extreme  

Example: `"insult": 2`
Do NOT output `"insult": 2.0` or `"insult": "2"`

=========================
TARGETS (BOOLEAN FLAGS)
=========================
Each target field must be a **boolean** (`true` or `false`) and only set to true if that group is explicitly targeted.

=========================
JSON SCHEMA (MUST MATCH EXACTLY)
=========================
{
  "overall": {
    "score": 0.00,
    "label": "neutral"
  },
  "facets": {
    "sentiment": 0,
    "respect": 0,
    "insult": 0,
    "humiliate": 0,
    "status": 0,
    "dehumanize": 0,
    "violence": 0,
    "genocide": 0,
    "attack_defend": 0,
    "hatespeech": 0
  },
  "targets": {
    "target_race_asian": false,
    "target_race_black": false,
    "target_race_latinx": false,
    "target_race_middle_eastern": false,
    "target_race_native_american": false,
    "target_race_pacific_islander": false,
    "target_race_white": false,
    "target_race_other": false,
    "target_religion_atheist": false,
    "target_religion_buddhist": false,
    "target_religion_christian": false,
    "target_religion_hindu": false,
    "target_religion_jewish": false,
    "target_religion_mormon": false,
    "target_religion_muslim": false,
    "target_religion_other": false,
    "target_origin_immigrant": false,
    "target_origin_migrant_worker": false,
    "target_origin_specific_country": false,
    "target_origin_undocumented": false,
    "target_origin_other": false,
    "target_gender_men": false,
    "target_gender_non_binary": false,
    "target_gender_transgender_men": false,
    "target_gender_transgender_unspecified": false,
    "target_gender_transgender_women": false,
    "target_gender_women": false,
    "target_gender_other": false,
    "target_sexuality_bisexual": false,
    "target_sexuality_gay": false,
    "target_sexuality_lesbian": false,
    "target_sexuality_straight": false,
    "target_sexuality_other": false,
    "target_age_children": false,
    "target_age_teenagers": false,
    "target_age_young_adults": false,
    "target_age_middle_aged": false,
    "target_age_seniors": false,
    "target_age_other": false,
    "target_disability_physical": false,
    "target_disability_cognitive": false,
    "target_disability_neurological": false,
    "target_disability_visually_impaired": false,
    "target_disability_hearing_impaired": false,
    "target_disability_unspecific": false,
    "target_disability_other": false
  }
}

=========================
TEXT TO ANALYZE
=========================
{text}```

# Benchmarks:

Overall Assessment:
Label: We'll calculate the Macro F1-Score and Accuracy for the overall.label field ("hateful", "neutral", "supportive")

Facet Analysis (Ordinal):
Accuracy: We'll calculate the Mean Absolute Error (MAE) for each of the 10 facets by comparing our gold integer values to the model's predicted integer values.

Target Identification (Multi-Label):
Accuracy:

In [2]:
import pandas as pd

df = pd.read_parquet("hf://datasets/ucberkeley-dlab/measuring-hate-speech/measuring-hate-speech.parquet")

In [7]:
column_names_list = df.columns.tolist()
print(column_names_list)

['comment_id', 'annotator_id', 'platform', 'sentiment', 'respect', 'insult', 'humiliate', 'status', 'dehumanize', 'violence', 'genocide', 'attack_defend', 'hatespeech', 'hate_speech_score', 'text', 'infitms', 'outfitms', 'annotator_severity', 'std_err', 'annotator_infitms', 'annotator_outfitms', 'hypothesis', 'target_race_asian', 'target_race_black', 'target_race_latinx', 'target_race_middle_eastern', 'target_race_native_american', 'target_race_pacific_islander', 'target_race_white', 'target_race_other', 'target_race', 'target_religion_atheist', 'target_religion_buddhist', 'target_religion_christian', 'target_religion_hindu', 'target_religion_jewish', 'target_religion_mormon', 'target_religion_muslim', 'target_religion_other', 'target_religion', 'target_origin_immigrant', 'target_origin_migrant_worker', 'target_origin_specific_country', 'target_origin_undocumented', 'target_origin_other', 'target_origin', 'target_gender_men', 'target_gender_non_binary', 'target_gender_transgender_men',